# Hierarchical Latent Dirichlet Allocation (HLDA) Topic Modeling Algorithm Introduction

### Joseph Jinn and Keith VanderLinden

### Plate Notation for the HLDA Algorithm:

### Statistical Formula for Calculating the HLDA algorithm):

### Pseudocode for the HLDA algorithm:

## A Simplified Hierarchical Latent Dirichlet Allocation Topic Modeling Algorithm Example:

### Completion of the FIRST iteration of the HLDA algorithm:

Rinse and repeat.

## Resources Referenced: